In [23]:
import pandas as pd

In [24]:
#Cleaned data, removed "", changed separator
#should try with thi instead: engine="python", sep=',', quotechar='"', error_bad_lines=False)

In [25]:
df = pd.read_csv('/content/SMSSpamCollection.csv', sep='¤', engine='python')

In [26]:
df.head()

,CATEGORY,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [27]:
df[222:250]

,CATEGORY,SMS
222,ham,There is os called ubandu which will run witho...
223,ham,"Sorry, I'll call later"
224,ham,U say leh... Of course nothing happen lar. Not...
225,spam,"500 New Mobiles from 2004, MUST GO! Txt: NOKIA..."
226,ham,Would really appreciate if you call me. Just n...
227,spam,Will u meet ur dream partner soon? Is ur caree...
228,ham,Hey company elama po mudyadhu.
229,ham,Life is more strict than teacher... Bcoz Teach...
230,ham,Dear good morning now only i am up
231,ham,Get down in gandhipuram and walk to cross cut ...


In [28]:
print('NR OF SAMPLES:', len(df))

NR OF SAMPLES: 5574


In [29]:
#check for empty values ....   
#df.isnull().sum()
#df.info()
# - invalid values?
#df.dropna(inplace=True)  # will remove rows with NA

In [30]:
X = df['SMS']
y = df['CATEGORY']

In [31]:
y.unique()

array(['ham', 'spam', 'spam '], dtype=object)

In [32]:
y = y.str.strip()
y.unique()

array(['ham', 'spam'], dtype=object)

In [33]:
y[y == 'spam'] = 1
y[y == 'ham'] = 0
y[:5]

0    0
1    0
2    1
3    0
4    0
Name: CATEGORY, dtype: object

In [34]:
import numpy as np
y = np.asarray(y).astype(int)

In [35]:
print('***** Nr of characters the longest content is: ',
max([len(x) for x in X]))

***** Nr of characters the longest content is:  910


In [36]:
#TRY TO VISUALIZE THE DATA ! ! !

In [37]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
total_words = len(tokenizer.word_index) + 1
total_words

9009

In [38]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [39]:
X = tokenizer.texts_to_sequences(X)
X[:3]

[[49,
  471,
  4434,
  842,
  755,
  658,
  64,
  8,
  1327,
  88,
  123,
  351,
  1328,
  148,
  2996,
  1329,
  67,
  58,
  4435,
  144],
 [46, 336, 1499, 472, 6, 1940],
 [47,
  489,
  8,
  19,
  4,
  797,
  901,
  2,
  176,
  1941,
  1105,
  659,
  1942,
  2331,
  261,
  2332,
  71,
  1941,
  2,
  1943,
  2,
  337,
  489,
  555,
  960,
  73,
  391,
  174,
  660,
  392,
  2997]]

In [40]:
# pad content based on the longest SMS...
max_SMS_len = max(len(x) for x in X)
print('*****Max words in SMS:', max_SMS_len)

from tensorflow.keras.preprocessing.sequence import pad_sequences
X = pad_sequences(X, maxlen=100)
X[2]

*****Max words in SMS: 189


array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,   47,  489,    8,   19,    4,  797,  901,    2,
        176, 1941, 1105,  659, 1942, 2331,  261, 2332,   71, 1941,    2,
       1943,    2,  337,  489,  555,  960,   73,  391,  174,  660,  392,
       2997], dtype=int32)

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 22)
print('***** Shape of training samples: ', X_train.shape)
print('***** Shape of testing samples: ', X_test.shape)

***** Shape of training samples:  (4459, 100)
***** Shape of testing samples:  (1115, 100)


In [42]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Embedding(9009, 128)) # 48998 distinct words, 128 neurons (columns in Excel)
# model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(100, dropout=0.4, recurrent_dropout=0.4))
# model.add(Dense(units=64, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))

In [43]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [44]:
model.fit(X_train, y_train,
          batch_size=32,
          epochs=3,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/3
140/140 [==============================] - 46s 304ms/step - loss: 0.1693 - accuracy: 0.9415 - val_loss: 0.0590 - val_accuracy: 0.9830
Epoch 2/3
140/140 [==============================] - 35s 246ms/step - loss: 0.0307 - accuracy: 0.9915 - val_loss: 0.0566 - val_accuracy: 0.9857
Epoch 3/3
140/140 [==============================] - 34s 244ms/step - loss: 0.0081 - accuracy: 0.9984 - val_loss: 0.0538 - val_accuracy: 0.9892


In [45]:
score, acc = model.evaluate(X_test, y_test,
                            batch_size=32,
                            verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

35/35 - 1s - loss: 0.0538 - accuracy: 0.9892 - 1s/epoch - 30ms/step
Test score: 0.05375980958342552
Test accuracy: 0.9892376661300659


In [46]:
new_SMS = "I am tired of doing the same things every day"
new_SMS = tokenizer.texts_to_sequences([new_SMS])
#print(new_SMS)
new_SMS = pad_sequences(new_SMS, maxlen=100)

In [47]:
result = model.predict(new_SMS)
result

array([[0.00021592]], dtype=float32)

In [48]:
new_SPAM = "500 fresh burguers MUST GO! Txt: KING to No: 220022"
new_SPAM = tokenizer.texts_to_sequences([new_SPAM])
#print(new_SPAM)
new_SPAM = pad_sequences(new_SPAM, maxlen=100)
result = model.predict(new_SPAM)
print(result)

[[0.37426776]]


In [49]:
model.save("SpamHam.h5")